In [1]:
import pandas as pd
import datetime


#### Covid data collected from https://github.com/microsoft/Bing-COVID-19-Data/tree/master/data


In [3]:
#### custom funciton for formatting the date as same as the covid dataset
def date_format(x):
    try:
        x = str(x)
        date = x.split()[0]
        return datetime.datetime.strptime(date, "%m/%d/%Y").strftime("%d/%m/%Y")
    except:
        print("nan found")

In [4]:
covid_data =  pd.read_csv("covid.csv")
passenger_data = pd.read_csv("passenger_data.csv")
country_data = pd.read_csv("country_data.csv")

C:\Users\Asus\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (15) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [5]:
covid_df = pd.DataFrame(covid_data)
passenger_df = pd.DataFrame(passenger_data)
country_df = pd.DataFrame(country_data)


#Removing unnecessary columns

covid_df = covid_df.iloc[:, 1:]
country_df = country_df.iloc[:, 1:]
passenger_df = passenger_df.iloc[:, 1:]

In [6]:
covid_df.head()


covid_df = covid_df.drop(covid_df[pd.notna(covid_df.AdminRegion1)].index)

#removing the unnecessary admin region1 and 2 data, we only need the country data here

#pd.notna(covid_df.AdminRegion1) means all the row where admin region1 is not null 
# and we drop them
# where admin region1 not null dropped, all the notnull value of adminregion2 automatically dropped
# as there is not admin region2 notnull value without admin region1 value

In [7]:
# Keeping the necessary column for analysis
covid_df_n = covid_df[['Updated', 'Confirmed', 'ConfirmedChange', 'Deaths',
       'DeathsChange', 'Recovered', 'RecoveredChange', 'Country_Region']]
covid_df_n = covid_df_n[covid_df_n["Country_Region"] != "Worldwide"]
covid_df_n = covid_df_n.rename(columns ={"Country_Region": "Country Name"})
#covid_df.columns
covid_df_n.head()

,Updated,Confirmed,ConfirmedChange,Deaths,DeathsChange,Recovered,RecoveredChange,Country Name
367,02/24/2020,1,NaN,0.0,NaN,NaN,NaN,Afghanistan
368,02/25/2020,1,0.0,0.0,0.0,NaN,NaN,Afghanistan
369,02/26/2020,1,0.0,0.0,0.0,NaN,NaN,Afghanistan
370,02/27/2020,1,0.0,0.0,0.0,NaN,NaN,Afghanistan
371,02/28/2020,1,0.0,0.0,0.0,NaN,NaN,Afghanistan


In [8]:
covid_df_n["Updated"] = covid_df_n["Updated"].apply(date_format)
#only one null value found, otherwise all date fomat is corrected to desired format

In [9]:
passenger_df

,Country Name,Updated,number_of_flights
0,Algeria,02/04/2020,3
1,Algeria,03/04/2020,2
2,Algeria,04/04/2020,4
3,Algeria,05/04/2020,3
4,Algeria,06/04/2020,2
...,...,...,...
35919,Zambia,26/09/2020,8
35920,Zambia,27/09/2020,6
35921,Zambia,28/09/2020,7
35922,Zambia,29/09/2020,7


In [10]:
country_df = country_df.rename(columns ={"Date": "Updated"})

country_df = country_df[['Country Name', 'gdp', 'land_area', 'population', 'Income Level']]
country_df.head()

,Country Name,gdp,land_area,population,Income Level
0,Afghanistan,1.929110e+10,652860.0,38041754.0,Low income
1,Afghanistan,1.929110e+10,652860.0,38041754.0,Low income
2,Afghanistan,1.929110e+10,652860.0,38041754.0,Low income
3,Afghanistan,1.929110e+10,652860.0,38041754.0,Low income
4,Afghanistan,1.929110e+10,652860.0,38041754.0,Low income


In [11]:
country_df.shape

(2766, 5)

In [12]:
country_df = country_df.groupby(['Country Name'], as_index = False).min()

In [13]:
# Creating a new dataframe merging all the three datasets
new_df = covid_df_n.merge(passenger_df, on=["Country Name", "Updated"], how="outer").merge(country_df, on=["Country Name"], how="outer")
new_df.tail(1000)

,Updated,Confirmed,ConfirmedChange,Deaths,DeathsChange,Recovered,RecoveredChange,Country Name,number_of_flights,gdp,land_area,population,Income Level
76209,11/11/2020,NaN,NaN,NaN,NaN,NaN,NaN,Czech Republic,55.0,NaN,NaN,NaN,NaN
76210,12/11/2020,NaN,NaN,NaN,NaN,NaN,NaN,Czech Republic,62.0,NaN,NaN,NaN,NaN
76211,13/11/2020,NaN,NaN,NaN,NaN,NaN,NaN,Czech Republic,69.0,NaN,NaN,NaN,NaN
76212,14/11/2020,NaN,NaN,NaN,NaN,NaN,NaN,Czech Republic,51.0,NaN,NaN,NaN,NaN
76213,15/11/2020,NaN,NaN,NaN,NaN,NaN,NaN,Czech Republic,55.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
77204,26/09/2020,NaN,NaN,NaN,NaN,NaN,NaN,Macau,8.0,NaN,NaN,NaN,NaN
77205,27/09/2020,NaN,NaN,NaN,NaN,NaN,NaN,Macau,19.0,NaN,NaN,NaN,NaN
77206,28/09/2020,NaN,NaN,NaN,NaN,NaN,NaN,Macau,16.0,NaN,NaN,NaN,NaN
77207,29/09/2020,NaN,NaN,NaN,NaN,NaN,NaN,Macau,14.0,NaN,NaN,NaN,NaN


In [14]:
new_df.to_csv("master_data.csv")

In [15]:
new_df.shape

(77209, 13)

In [16]:
new_df.tail()

,Updated,Confirmed,ConfirmedChange,Deaths,DeathsChange,Recovered,RecoveredChange,Country Name,number_of_flights,gdp,land_area,population,Income Level
77204,26/09/2020,NaN,NaN,NaN,NaN,NaN,NaN,Macau,8.0,NaN,NaN,NaN,NaN
77205,27/09/2020,NaN,NaN,NaN,NaN,NaN,NaN,Macau,19.0,NaN,NaN,NaN,NaN
77206,28/09/2020,NaN,NaN,NaN,NaN,NaN,NaN,Macau,16.0,NaN,NaN,NaN,NaN
77207,29/09/2020,NaN,NaN,NaN,NaN,NaN,NaN,Macau,14.0,NaN,NaN,NaN,NaN
77208,30/09/2020,NaN,NaN,NaN,NaN,NaN,NaN,Macau,40.0,NaN,NaN,NaN,NaN


In [17]:
covid_data.shape

(1381816, 16)

In [18]:
new_df[new_df["Country Name"] == "Argentina"].tail(50)

,Updated,Confirmed,ConfirmedChange,Deaths,DeathsChange,Recovered,RecoveredChange,Country Name,number_of_flights,gdp,land_area,population,Income Level
3036,13/02/2020,NaN,NaN,NaN,NaN,NaN,NaN,Argentina,209.0,4.454452e+11,2736690.0,44938712.0,Upper middle income
3037,14/02/2020,NaN,NaN,NaN,NaN,NaN,NaN,Argentina,211.0,4.454452e+11,2736690.0,44938712.0,Upper middle income
3038,15/02/2020,NaN,NaN,NaN,NaN,NaN,NaN,Argentina,222.0,4.454452e+11,2736690.0,44938712.0,Upper middle income
3039,16/02/2020,NaN,NaN,NaN,NaN,NaN,NaN,Argentina,199.0,4.454452e+11,2736690.0,44938712.0,Upper middle income
3040,17/02/2020,NaN,NaN,NaN,NaN,NaN,NaN,Argentina,211.0,4.454452e+11,2736690.0,44938712.0,Upper middle income
3041,18/02/2020,NaN,NaN,NaN,NaN,NaN,NaN,Argentina,209.0,4.454452e+11,2736690.0,44938712.0,Upper middle income
3042,19/02/2020,NaN,NaN,NaN,NaN,NaN,NaN,Argentina,201.0,4.454452e+11,2736690.0,44938712.0,Upper middle income
3043,20/02/2020,NaN,NaN,NaN,NaN,NaN,NaN,Argentina,204.0,4.454452e+11,2736690.0,44938712.0,Upper middle income
3044,21/02/2020,NaN,NaN,NaN,NaN,NaN,NaN,Argentina,219.0,4.454452e+11,2736690.0,44938712.0,Upper middle income
3045,22/02/2020,NaN,NaN,NaN,NaN,NaN,NaN,Argentina,227.0,4.454452e+11,2736690.0,44938712.0,Upper middle income
